In [2]:
# Importiere die erforderlichen Bibliotheken
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Lade die output.csv Datei
data = pd.read_csv("output.csv")

# Daten vorverarbeiten
features = data[['contour number', 'aspect ratio', 'extent', 'Blue', 'Green', 'Red', 'Hue']]
target = data['class']

# Teile die Daten in Trainings- und Testsets auf
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=42)

# Kodiere die Zielvariable
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_target)
test_labels = label_encoder.transform(test_target)

# Lade das vortrainierte VGG16-Modell
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

# Füge benutzerdefinierte Klassifikationsschichten oben auf dem Basismodell hinzu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Erstelle das Transfer-Learning-Modell
model = Model(inputs=base_model.input, outputs=predictions)

# Kompiliere das Modell
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Trainiere das Modell
model.fit(train_features, train_labels, epochs=10, validation_data=(test_features, test_labels))

# Evaluieren des Modells
test_loss, test_acc = model.evaluate(test_features, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")


ModuleNotFoundError: No module named 'tensorflow'